# Tensor de entrada

In [11]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
import json 

# Cargar los datos del archivo CSV
df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')

# Obtener fechas únicas
fechas_unicas = df_solcast['Date'].unique()

# Lista para guardar los tensores de cada día
tensors_dias = []
# Diccionario para mapear fechas a índices en tensors_dias
fechas_indices = {}

# Escalador MinMax
scaler = MinMaxScaler()

for fecha in fechas_unicas:
    row_day_solcast = df_solcast.loc[df_solcast['Date'] == fecha]
    row_day_fronius = df_SFV.loc[df_SFV['Date'] == fecha]

    # Verificar que el día tiene 288 registros
    if len(row_day_solcast) == 288 and len(row_day_fronius) == 288:
        # Procesamiento normal...
        # Resetear y ordenar por hora
        row_day_solcast = row_day_solcast.sort_values(by='Time').reset_index(drop=True)
        row_day_fronius = row_day_fronius.reset_index(drop=True)
    
        # Crear DataFrame para el día actual
        df_dia = pd.DataFrame({
            'Time': row_day_solcast['Time'],
            'GHI': row_day_solcast['GHI'],
            'air_temp': row_day_solcast['air_temp'],
            'cloud_opacity': row_day_solcast['cloud_opacity'],
            'Producción_fotovoltaica_SFV': row_day_fronius['Producción_fotovoltaica_SFV'],
            'clearsky_ghi': row_day_solcast['clearsky_ghi'],
        })
    
        # Normalizar los datos
        df_dia[['GHI', 'Producción_fotovoltaica_SFV', 'clearsky_ghi', 'cloud_opacity', 'air_temp']] = scaler.fit_transform(df_dia[['GHI', 'Producción_fotovoltaica_SFV', 'clearsky_ghi', 'cloud_opacity', 'air_temp']])
    
        # Convertir a tensor y cambiar forma
        ghi_tensor = torch.tensor(df_dia['GHI'].values).view(-1, 1)
        air_temp_tensor = torch.tensor(df_dia['air_temp'].values).view(-1, 1)
        cloud_opacity_tensor = torch.tensor(df_dia['cloud_opacity'].values).view(-1, 1)
    
        # Concatenar y cambiar forma a [1, 288, 3]
        dia_tensor = torch.cat((ghi_tensor, air_temp_tensor, cloud_opacity_tensor), dim=1).unsqueeze(0)
    
        # Agregar el tensor del día a la lista
        tensors_dias.append(dia_tensor)
        # Agregar la fecha al diccionario
        fechas_indices[fecha] = len(tensors_dias) - 1

# Verificar si hay días para concatenar
if tensors_dias:
    # Concatenar todos los tensores de días en un tensor [n, 288, 3]
    tensor_final = torch.cat(tensors_dias, dim=0)
    print(tensor_final.shape)  # Salida esperada: torch.Size([n, 288, 3])
    # Almacenar el tensor en un archivo
    np.save('input_tensor.npy', tensor_final)
    with open('indice_fechas_input.json', 'w') as fp:
        json.dump(fechas_indices, fp)
else:
    print("No hay suficientes datos para crear el tensor final.")


torch.Size([180, 288, 3])


## Tensor SFV

In [13]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler

# Cargar los datos del archivo CSV
df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')

# Obtener fechas únicas
fechas_unicas = df_solcast['Date'].unique()

# Lista para guardar los tensores de cada día de la producción fotovoltaica
tensors_SFV = []
# Diccionario para mapear fechas a índices en tensors_SFV
indice_fechas_SFV = {}

# Escalador MinMax
scaler = MinMaxScaler()

for fecha in fechas_unicas:
    row_day_solcast = df_solcast.loc[df_solcast['Date'] == fecha]
    row_day_fronius = df_SFV.loc[df_SFV['Date'] == fecha]

    # Verificar que el día tiene 288 registros
    if len(row_day_solcast) == 288 and len(row_day_fronius) == 288:
        # Resetear y ordenar por hora
        row_day_solcast = row_day_solcast.sort_values(by='Time').reset_index(drop=True)
        row_day_fronius = row_day_fronius.reset_index(drop=True)
    
        # Crear DataFrame para el día actual
        df_dia = pd.DataFrame({
            'Time': row_day_solcast['Time'],
            'GHI': row_day_solcast['GHI'],
            'air_temp': row_day_solcast['air_temp'],
            'cloud_opacity': row_day_solcast['cloud_opacity'],
            'Producción_fotovoltaica_SFV': row_day_fronius['Producción_fotovoltaica_SFV'],
            'clearsky_ghi': row_day_solcast['clearsky_ghi'],
        })
    
        # Normalizar los datos de interés
        df_dia['Producción_fotovoltaica_SFV'] = scaler.fit_transform(df_dia[['Producción_fotovoltaica_SFV']])
    
        # Convertir la producción a tensor y cambiar forma
        produccion_tensor = torch.tensor(df_dia['Producción_fotovoltaica_SFV'].values).view(1, -1, 1)
        
        # Agregar el tensor del día a la lista
        tensors_SFV.append(produccion_tensor)
        # Registrar la fecha en el diccionario de índices
        indice_fechas_SFV[fecha] = len(tensors_SFV) - 1

# Concatenar todos los tensores de días en un tensor [n, 288, 1]
if tensors_SFV:
    tensor_final = torch.cat(tensors_SFV, dim=0)
    print(tensor_final.shape)  # Salida esperada: torch.Size([n, 288, 1])
    # Almacenar el tensor en un archivo
    np.save('tensor_SFV.npy', tensor_final)
    # Almacenar el mapeo de fechas a índices como JSON
    with open('indice_fechas_SFV.json', 'w') as fp:
        json.dump(indice_fechas_SFV, fp)
else:
    print("No hay suficientes datos para crear el tensor final.")


torch.Size([180, 288, 1])


# Tensores con fecha  📆


In [24]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
import json 

# Cargar los datos del archivo CSV
df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')

# crear columna Date* a partir de la columna Date
df_solcast ['Date*'] = df_solcast ['Date'] 

# quitar caracteres especiales y espacios
df_solcast ['Date*'] = df_solcast ['Date*'].str.replace(' ', '')
df_solcast ['Date*'] = df_solcast ['Date*'].str.replace('-', '')
print(df_solcast ['Date*'])


0         20230101
1         20230101
2         20230101
3         20230101
4         20230101
            ...   
105403    20240101
105404    20240101
105405    20240101
105406    20240101
105407    20240102
Name: Date*, Length: 105408, dtype: object


In [25]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
import json 

# Cargar los datos del archivo CSV
# df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
# df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')

# Obtener fechas únicas
fechas_unicas = df_solcast['Date'].unique()

# Lista para guardar los tensores de cada día
tensors_dias = []
# Diccionario para mapear fechas a índices en tensors_dias
fechas_indices = {}

# Escalador MinMax
scaler = MinMaxScaler()

for fecha in fechas_unicas:
    row_day_solcast = df_solcast.loc[df_solcast['Date'] == fecha]
    row_day_fronius = df_SFV.loc[df_SFV['Date'] == fecha]

    # Verificar que el día tiene 288 registros
    if len(row_day_solcast) == 288 and len(row_day_fronius) == 288:
        # Procesamiento normal...
        # Resetear y ordenar por hora
        row_day_solcast = row_day_solcast.sort_values(by='Time').reset_index(drop=True)
        row_day_fronius = row_day_fronius.reset_index(drop=True)
    
        # Crear DataFrame para el día actual
        df_dia = pd.DataFrame({
            'Time': row_day_solcast['Time'],
            'GHI': row_day_solcast['GHI'],
            'air_temp': row_day_solcast['air_temp'],
            'cloud_opacity': row_day_solcast['cloud_opacity'],
            'Producción_fotovoltaica_SFV': row_day_fronius['Producción_fotovoltaica_SFV'],
            'clearsky_ghi': row_day_solcast['clearsky_ghi'],
            'date': row_day_solcast['Date*']
        })
    
        # Normalizar los datos
        df_dia[['GHI', 'Producción_fotovoltaica_SFV', 'clearsky_ghi', 'cloud_opacity', 'air_temp', 'date']] = scaler.fit_transform(df_dia[['GHI', 'Producción_fotovoltaica_SFV', 'clearsky_ghi', 'cloud_opacity', 'air_temp', 'date']])
    
        # Convertir a tensor y cambiar forma
        ghi_tensor = torch.tensor(df_dia['GHI'].values).view(-1, 1)
        air_temp_tensor = torch.tensor(df_dia['air_temp'].values).view(-1, 1)
        cloud_opacity_tensor = torch.tensor(df_dia['cloud_opacity'].values).view(-1, 1)
        date_tensor = torch.tensor(df_dia['date'].values).view(-1, 1)
    
        # Concatenar y cambiar forma a [1, 288, 3]
        dia_tensor = torch.cat((ghi_tensor, air_temp_tensor, cloud_opacity_tensor, date_tensor), dim=1).unsqueeze(0)
    
        # Agregar el tensor del día a la lista
        tensors_dias.append(dia_tensor)
        # Agregar la fecha al diccionario
        fechas_indices[fecha] = len(tensors_dias) - 1

# Verificar si hay días para concatenar
if tensors_dias:
    # Concatenar todos los tensores de días en un tensor [n, 288, 3]
    tensor_final = torch.cat(tensors_dias, dim=0)
    print(tensor_final.shape)  # Salida esperada: torch.Size([n, 288, 3])
    # Almacenar el tensor en un archivo
    np.save('input_tensor.npy', tensor_final)
    with open('indice_fechas_input.json', 'w') as fp:
        json.dump(fechas_indices, fp)
else:
    print("No hay suficientes datos para crear el tensor final.")


torch.Size([180, 288, 4])


## Tensor SFV

In [26]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler

# Cargar los datos del archivo CSV
df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')

# crear columna Date* a partir de la columna Date
df_SFV ['Date**'] = df_SFV ['Date'] 

# quitar caracteres especiales y espacios
df_SFV ['Date**'] = df_SFV ['Date**'].str.replace(' ', '')
df_SFV ['Date**'] = df_SFV ['Date**'].str.replace('-', '')
print(df_SFV ['Date**'])

# Obtener fechas únicas
fechas_unicas = df_solcast['Date'].unique()

# Lista para guardar los tensores de cada día de la producción fotovoltaica
tensors_SFV = []
# Diccionario para mapear fechas a índices en tensors_SFV
indice_fechas_SFV = {}

# Escalador MinMax
scaler = MinMaxScaler()

for fecha in fechas_unicas:
    row_day_solcast = df_solcast.loc[df_solcast['Date'] == fecha]
    row_day_fronius = df_SFV.loc[df_SFV['Date'] == fecha]

    # Verificar que el día tiene 288 registros
    if len(row_day_solcast) == 288 and len(row_day_fronius) == 288:
        # Resetear y ordenar por hora
        row_day_solcast = row_day_solcast.sort_values(by='Time').reset_index(drop=True)
        row_day_fronius = row_day_fronius.reset_index(drop=True)
    
        # Crear DataFrame para el día actual
        df_dia = pd.DataFrame({
            'Time': row_day_solcast['Time'],
            'GHI': row_day_solcast['GHI'],
            'air_temp': row_day_solcast['air_temp'],
            'cloud_opacity': row_day_solcast['cloud_opacity'],
            'Producción_fotovoltaica_SFV': row_day_fronius['Producción_fotovoltaica_SFV'],
            'date': row_day_fronius['Date**'],
            'clearsky_ghi': row_day_solcast['clearsky_ghi'],
        })
    
        # Normalizar los datos de interés
        df_dia['Producción_fotovoltaica_SFV', 'date'] = scaler.fit_transform(df_dia[['Producción_fotovoltaica_SFV', 'date']])
    
        # Convertir la producción a tensor y cambiar forma
        produccion_tensor = torch.tensor(df_dia['Producción_fotovoltaica_SFV', 'date'].values).view(1, -1, 1)
        
        # Agregar el tensor del día a la lista
        tensors_SFV.append(produccion_tensor)
        # Registrar la fecha en el diccionario de índices
        indice_fechas_SFV[fecha] = len(tensors_SFV) - 1

# Concatenar todos los tensores de días en un tensor [n, 288, 1]
if tensors_SFV:
    tensor_final = torch.cat(tensors_SFV, dim=0)
    print(tensor_final.shape)  # Salida esperada: torch.Size([n, 288, 1])
    # Almacenar el tensor en un archivo
    np.save('tensor_SFV.npy', tensor_final)
    # Almacenar el mapeo de fechas a índices como JSON
    with open('indice_fechas_SFV.json', 'w') as fp:
        json.dump(indice_fechas_SFV, fp)
else:
    print("No hay suficientes datos para crear el tensor final.")


0        20230101
1        20230101
2        20230101
3        20230101
4        20230101
           ...   
52123    20230630
52124    20230630
52125    20230630
52126    20230630
52127    20230630
Name: Date**, Length: 52128, dtype: object


ValueError: Expected a 1D array, got an array with shape (288, 2)

In [27]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
import json  # Importa el módulo json para la serialización

# Cargar los datos del archivo CSV
df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')

# crear columna Date** a partir de la columna Date
df_SFV['Date**'] = df_SFV['Date'].str.replace(' ', '').replace('-', '', regex=True)

# Obtener fechas únicas
fechas_unicas = df_solcast['Date'].unique()

# Lista para guardar los tensores de cada día de la producción fotovoltaica
tensors_SFV = []

# Diccionario para mapear fechas a índices en tensors_SFV
indice_fechas_SFV = {}

# Escalador MinMax
scaler = MinMaxScaler()

for fecha in fechas_unicas:
    row_day_solcast = df_solcast[df_solcast['Date'] == fecha]
    row_day_fronius = df_SFV[df_SFV['Date'] == fecha]

    if len(row_day_solcast) == 288 and len(row_day_fronius) == 288:
        row_day_solcast = row_day_solcast.sort_values(by='Time').reset_index(drop=True)
        row_day_fronius = row_day_fronius.reset_index(drop=True)
    
        df_dia = pd.DataFrame({
            'Time': row_day_solcast['Time'],
            'GHI': row_day_solcast['GHI'],
            'air_temp': row_day_solcast['air_temp'],
            'cloud_opacity': row_day_solcast['cloud_opacity'],
            'Producción_fotovoltaica_SFV': row_day_fronius['Producción_fotovoltaica_SFV'],
            'date': row_day_fronius['Date**'],
            'clearsky_ghi': row_day_solcast['clearsky_ghi'],
        })
        
        # Normalizar los datos de interés
        normalized_values = scaler.fit_transform(df_dia[['Producción_fotovoltaica_SFV', 'date']])
        df_dia[['Producción_fotovoltaica_SFV', 'date']] = normalized_values
        
        # Convertir la producción a tensor y cambiar forma
        produccion_tensor = torch.tensor(normalized_values).view(1, -1, 2)  # Ajusta esta línea según la estructura deseada
        
        tensors_SFV.append(produccion_tensor)
        indice_fechas_SFV[fecha] = len(tensors_SFV) - 1

# Concatenar todos los tensores de días en un tensor [n, 288, 1]
if tensors_SFV:
    tensor_final = torch.cat(tensors_SFV, dim=0)
    print(tensor_final.shape)  # Salida esperada: torch.Size([n, 288, 1])
    # Almacenar el tensor en un archivo
    np.save('tensor_SFV.npy', tensor_final)
    # Almacenar el mapeo de fechas a índices como JSON
    with open('indice_fechas_SFV.json', 'w') as fp:
        json.dump(indice_fechas_SFV, fp)
else:
    print("No hay suficientes datos para crear el tensor final.")



torch.Size([180, 288, 2])
